In [1]:
import datasets
import pandas as pd
from IPython.display import display, HTML
from datasets import ClassLabel
from transformers import BertTokenizerFast
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments,AutoTokenizer,RobertaConfig
from transformers import EncoderDecoderModel
from transformers import BertTokenizer
from rouge import Rouge
import numpy as np
from hazm import *

2025-01-09 09:18:53.014910: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-09 09:18:53.015141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-09 09:18:53.565574: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-09 09:18:54.896914: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-09 09:19:01.998930: W tensorflow/compiler/tf2

In [49]:
train_data = datasets.load_dataset(
    "pn_summary", split="train", download_mode="force_redownload"
)

pn_summary.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

pn_summary.zip:   0%|          | 0.00/89.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/82022 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5592 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5593 [00:00<?, ? examples/s]

In [50]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/roberta-fa-zwnj-base")

In [51]:
normalizer = Normalizer()

def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    for i in range(len(batch['article'])):
      batch['article'][i]=normalizer.normalize(batch['article'][i])
      batch['summary'][i]=normalizer.normalize(batch['summary'][i])

    inputs = tokenizer(
        batch["article"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
    )
    outputs = tokenizer(
        batch["summary"],
        padding="max_length",
        truncation=True,
        max_length=decoder_max_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()

    # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`.
    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [0 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [52]:
metric = Rouge()
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions

    labels = np.where(labels != 0, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Compute ROUGE scores
    result = metric.get_scores(decoded_preds, decoded_labels,avg=True)
    print(result)
    with open("result_roberta2roberta_temp.txt","a") as handler:
        handler.write(f'rouge-1 - recall: {result["rouge-1"]["r"]} - precision: {result["rouge-1"]["p"]} - fscore: {result["rouge-1"]["f"]}\n')
        handler.write(f'rouge-2 - recall: {result["rouge-2"]["r"]} - precision: {result["rouge-2"]["p"]} - fscore: {result["rouge-2"]["f"]}\n')
        handler.write(f'rouge-l - recall: {result["rouge-l"]["r"]} - precision: {result["rouge-l"]["p"]} - fscore: {result["rouge-l"]["f"]}\n\n')
    # Extract F-measure for each ROUGE score
    rouge_result = {
        "rouge1": result["rouge-1"]["f"],
        "rouge2": result["rouge-2"]["f"],
        "rougeL": result["rouge-l"]["f"],
    }

    return rouge_result

In [53]:
sample_size = 10000
encoder_max_length = 512
decoder_max_length = 128
#train_data = train_data.select(range(10000))
# batch_size = 16
batch_size = 4

In [ ]:
train_data = train_data.map(
        process_data_to_model_inputs,
        batched=True,
        batch_size=batch_size,
        remove_columns=['id', 'title', 'article', 'summary', 'category', 'categories', 'network', 'link'],
    )

In [ ]:
train_data.set_format(
        type="torch",
        columns=["input_ids", "attention_mask", "labels"],
    )

In [ ]:
val_data = datasets.load_dataset(
        "pn_summary", split="validation"
    )

In [ ]:
val_data = val_data.map(
        process_data_to_model_inputs,
        batched=True,
        batch_size=batch_size,
        remove_columns=['id', 'title', 'article', 'summary', 'category', 'categories', 'network', 'link'],
    )

In [ ]:
val_data.set_format(
        type="torch",
        columns=["input_ids", "attention_mask", "labels"],
    )

In [59]:
encoder_config = RobertaConfig.from_pretrained("HooshvareLab/roberta-fa-zwnj-base")
decoder_config = RobertaConfig.from_pretrained("HooshvareLab/roberta-fa-zwnj-base")

# Allow the decoder to use cross-attention
decoder_config.is_decoder = True
decoder_config.add_cross_attention = True

# Initialize the encoder-decoder model
roberta2roberta = EncoderDecoderModel.from_encoder_decoder_pretrained(
    "HooshvareLab/roberta-fa-zwnj-base", "HooshvareLab/roberta-fa-zwnj-base",
    encoder_config=encoder_config,
    decoder_config=decoder_config
)

roberta2roberta.save_pretrained("roberta2roberta_5")
roberta2roberta = EncoderDecoderModel.from_pretrained("roberta2roberta_5")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Some weights of RobertaModel were not initialized from the model checkpoint at HooshvareLab/roberta-fa-zwnj-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Some weights of RobertaForCausalLM were 

In [61]:
roberta2roberta.config.decoder_start_token_id = tokenizer.cls_token_id
roberta2roberta.config.eos_token_id = tokenizer.sep_token_id
roberta2roberta.config.pad_token_id = tokenizer.pad_token_id
roberta2roberta.config.vocab_size = roberta2roberta.config.encoder.vocab_size
roberta2roberta.config.max_length = 128
roberta2roberta.config.min_length = 0
roberta2roberta.config.no_repeat_ngram_size = 2
roberta2roberta.config.early_stopping = True
roberta2roberta.config.length_penalty = 2.0
roberta2roberta.config.num_beams = 5

In [62]:
roberta2roberta.config

EncoderDecoderConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "roberta2roberta_5",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_attn_implementation_autoset": true,
    "_name_or_path": "HooshvareLab/roberta-fa-zwnj-base",
    "add_cross_attention": true,
    "architectures": [
      "RobertaForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": 2,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_pr

In [63]:
training_args = Seq2SeqTrainingArguments(
        predict_with_generate=True,
        evaluation_strategy="steps",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        fp16=True,
        output_dir="roberta2roberta_model_5",
        logging_steps=2,
        save_steps=10000,
        eval_steps=10000,
        num_train_epochs=5,
        report_to="none",
        warmup_steps=1000,
    )

/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
        model=roberta2roberta,
        tokenizer=tokenizer,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_data,
        eval_dataset=val_data,
    )
trainer.train()

/tmp/ipykernel_1170/2773611530.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/opt/conda/lib/python3.11/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:629: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/opt/conda/lib/python3.11/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:649: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterio

Step,Training Loss,Validation Loss
